![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

cc_apps.replace("?", np.nan, inplace=True)
for col in cc_apps.select_dtypes(include=['number']).columns:
    cc_apps[col].fillna(cc_apps[col].mean(), inplace=True)
for col in cc_apps.select_dtypes(include=['object']).columns:
    cc_apps[col].fillna(cc_apps[col].mode()[0], inplace=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cc_apps.select_dtypes(include=['object']).columns:
    cc_apps[col] = le.fit_transform(cc_apps[col])

X = cc_apps.iloc[:, :-1]
y = cc_apps.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = LogisticRegression()
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)
print("accuracy:", accuracy_score(y_test, y_pred))
print("classification:", classification_report(y_test, y_pred))
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

accuracy: 0.8333333333333334
classification:               precision    recall  f1-score   support

           0       0.83      0.84      0.84        70
           1       0.84      0.82      0.83        68

    accuracy                           0.83       138
   macro avg       0.83      0.83      0.83       138
weighted avg       0.83      0.83      0.83       138

best parameters: {'C': 0.1, 'solver': 'liblinear'}
